## 经典数据表示

In [ ]:
浮点数（IEEE 754 normal/subnormal）


浮点规范 (bit for sign)(bits for exponent)(bits for mantissa)的方式，来近似实数. 采用二进制，它在数值上相等的实数值 = sign * [1 + fraction] * 2^(magnitude)  
  
fraction 是用 bits for mantissa 计算出来的。bits for mantissa，比如 FP64 规范 有 52 位bits, 二进制下这些 bits 的第1位到第52位分别代表 1/2 ,..., 1/2^52。累加后，加 1 (默认隐含)  
  
magnitude 是用 bits for exponent 计算出来的。bits for exponent，比如 FP64 规范 有 11 位bits, 二进制下这些 bits 可算出一个十进制指数. 减去偏置 bias（根据具体类型确定）  


Q1：fraction 为什么要加 1?  
A1：加 1 ，使得最终数值等于 1.xxx * 2^E 的形式，如果没有加 1，那么最终数值等于 0.xxx * 2^E 的形式。说明 mantissa 省略了科学计数法底数里的 1，节省出来的bits扩大了表示范围  
  
Q2：magnitude 为什么要减 bias?  
A2：简而言之是为了表示出 负指数，用以表示绝对值很小很小的数（规范的尺寸下限）。
比如 FP64 exponent部分有 11 位，按理最大(11个1)二进制表示2^11-1=2047。为了表示负指数，且正负平等，那么2047的一半1023作为bias，通过减去 bias 得到 负指数。
所以最大的正指数是 2046-1023 = 1023（全1的2047在 FP64 规范 里代表正无穷了），最小的负指数是 1-1023 = 1022（全0的exponent不符合normal 规约, 会触发subnormal 规约），
配合 1.fractional 的尾数，range上限是 1.11--1 * 2^1023, range下限是 0.00--1 * 2^-1022，由此确定了这个 FP64规范 的 尺寸(magnitude)

In [ ]:
规范 都分成3部分，符号位，指数和尾数。不同 规范 只不过是指数位和尾数位的长度不一样。

解析一个浮点数就5条规则

如果指数位全零，尾数位是全零，那就表示0
如果指数位全零，尾数位是非零，就表示一个很小的数（subnormal），计算方式 (−1)^signbit × 2^−126 × 0.fractionbits
如果指数位全是1，尾数位是全零，表示正负无穷
如果指数位全是1，尾数位是非零，表示不是一个数NAN
剩下的计算方式为 (−1)^signbit × 2^(exponentbits − bias) × 1.fractionbits

In [ ]:
FP64 double 8bytes 64bits
1位符号+11位指数+52位小数
range: 10^-308 至 10^308，precision：15-17位小数精度
IEEE 754定义，大多数C/C++系统里的double类型，x86CPU支持，GPU内支持但一般严重限制性能（除非一些专业卡）。一些科学计算（特别是迭代的）必须使用这个 FP64 不然累积误差过大

In [ ]:
FP32 float 4bytes 32bits
1位符号+8位指数+23位小数
range: 10^-38 至 10^38，precision：6-9位小数精度
IEEE 754定义，大多数C/C++系统里的float类型，x86CPU支持，GPU支持，是长期以来深度学习/神经网络的标准类型

理想情况下，训练和推理中涉及的数字都应该用 FP32 规范

In [ ]:
FP16 2bytes 16bits
1位符号+5位指数+10位小数
range: 10^-8 至 10^8，precision：4位小数精度
英伟达于2022年定义，C/C++ 不支持，x86 CPU 不支持，GPU良好支持。容易上溢（超出range上限）/下溢（比range下限更接近0）

理想情况下，训练和推理中涉及的数字都应该用 FP32 规范。但考虑到性能（FP16占据更少显存、速度更快），且精度没那么重要，实践中通常使用混合精度：
训练：
    使用 FP32 精度的weights作为 精确的 master weights。
    使用 FP16/BF6 等规范的权重作前向计算、后向传播（即梯度是 FP16/BF16）
    使用 FP16/BF16 的梯度去更新 FP32 的主权重
推理：
    FP16 规范的weights就有不错的效果


FP16 是一种独立的数据类型，从 FP32 转换到 FP16，需要显式类型转换，才能利用 FP16 的优势（内存带宽占用小，故更快的数据传输和执行）。

## 梯度上溢/下溢，以及解决办法

In [ ]:
由于 FP16 容易上溢(超出range上限)/下溢(比range下限更接近0)，若梯度发生了上下溢，那么上溢会导致梯度爆炸NaN，要么下溢导致权重无法更新(0梯度)。  

原生梯度下溢问题：
梯度缩放（loss scaling trick） 可以缓解。  
计算 loss 时，使用系数 a（一般是2的高次），乘以 original loss，得到 a_loss。对 a_loss 作反向传播后，得到的梯度 a_grad 也被同步缩放，所以要对 a_grad 除以 a。   

梯度缩放（loss scaling trick）过程中如果出现了 梯度上溢，即乘完 a 之后，梯度超出上限，这个时候动态梯度缩放机制会起作用：优化器会跳过当前的参数更新，然后减小系数 a。  
在下一次迭代中，使用缩小后的 a 作梯度缩放。如果没问题了，逐渐增大 a。   
  
  

  
原生梯度上溢问题：
原生梯度上溢/爆炸问题 有多种缓解办法：  
1. 混合精度训练，会涉及到上溢的操作，使用更大 magnitude 的 FP32 规范来算。
2. 梯度裁剪
3. 归一化层, batch norm 或 layer norm，归一化层可以有效稳定激活函数的范围，保持数值稳定
4. 更稳定的激活函数(relu > tanh/sigmoid)，更好的初始化策略、学习率调度等
5. 采用 派生的数据规范 比如 BF16 / TF32 等

## 派生数据表示

In [ ]:
截断 FP32，保留指数部分以保持 magnitude，但由于尾数部分被截断，所以损失 precision。

有两种截断方法：
TF32（从32位截断到19位，保留了FP32的magnitude，有 FP16 的precision）
BF16（从32位截断到16位，保留了FP32的magnitude，有 不如FP16的precision）

In [ ]:
TF32 19bits
1位符号+8位指数+10位小数，是 FP32 直接截断尾数部分（降低precision）至对齐FP16，保留指数部分（保留magnitude），总位数是19
range: 10^-38 至 10^38，precision：4位小数精度
英伟达 定义，C/C++ 不支持，x86 CPU 不支持，N卡A系列/cuda11 开始支持。

与 FP32 一样的范围，故上下溢问题比 FP16 要轻很多。精度和 FP16一样

TF32有一个优点：无需显式类型转换
它在cuda编译器/tensor core内部“深度黑箱式优化”，使得外部看来，它就是“低精度、高matmul性能的FP32”，输入输出都是 正经的FP32，只有内部部分操作时被转换了。
具体来说，
    1. 在外部接口看来，TF32和FP32都是torch.float32。
    2. 在模型执行底层，weights本身、以及非矩阵乘法matmul操作（比如激活函数等）都保持FP32。这些地方需要精度。
    3. 在进入tensor core执行的操作时（比如matmul），这些 FP32 会被转化成 TF32 进行计算，算完之后再转换成 FP32

In [ ]:
BF16 2bytes 16bits
1位符号+8位指数+7位小数，是 FP32 直接截断尾数部分（降低precision），保留指数部分（保留magnitude），使得总位数对齐 FP16
range: 10^-38 至 10^38，precision：3位小数精度
谷歌 定义，C/C++ 不支持，x86 CPU 不支持，N卡A系列/cuda11 开始支持。

与 FP32 一样的范围，故上下溢问题比 FP16 要轻很多。但是 BF16 的精度比 FP16 还低

过低的精度会导致收敛问题，包括但不限于误差在多次迭代中累积、小精度更新不稳定等。
解决办法包括 降低学习率以稳定收敛、增大batch_size以减少噪声、调整某些对精度敏感的layer、用稳定收敛的优化器、梯度裁剪等等。

BF16 是一种独立的数据类型，从 FP32 转换到 BF16，需要显式类型转换，才能利用 BF16 的优势（内存带宽占用小，故更快的数据传输和执行）。

In [ ]:
FP8 1byte 8bits  FP8有两种规范
E5M2：1位符号+5位指数+2位小数
range: 10^-8 至 10^8，precision：<1位小数精度，精度极低

E4M3：1位符号+4位指数+3位小数
range: 10^-4 至 10^4，precision：1-2位小数精度

英伟达 定义，C/C++ 不支持，x86 CPU 不支持，N卡H系列/cuda11 开始支持。


FP8 range小，精度低，但是数据宽度只有1字节，所以基本主要用于极大规模的 LLM，和推理加速（把训练好的权重和激活值量化到 FP8，可以大幅提升推理速度、降低部署成本）
其中 E5M2 的range更大，多用于梯度计算，和推理；E4M3 的range小，精度高一些，多用于 权重以及 精度敏感的操作。

不管是哪个规范的 FP8，其精度都很低。超低精度会导致 1. 训练难以收敛，2. 推理准确率低。为了弥补 FP8 的 低精度问题，需要用非常精密的 量化策略。
其中的核心技术是 动态量化。一般来说，FP8 规范必然搭配 动态量化 策略使用

## 量化/动态量化和量化感知

In [ ]:
量化：将"占用较多比特的浮点数"（源规范）转化成 "占用较少比特的浮点数or整数"（目标规范）。核心目的是 减少模型大小、加快数据传输、提高计算效率

quantization：源规范 --> 目标规范，dequantization：目标规范 --> 源规范

量化：
step1：数值映射，一般用 线性映射 的方法，比如
    源规范 FP32，SRC_min = -10^38, SRC_max = 10^38, SRC_zero = 0, 记为 O 
    目标规范 UINT8，TGT_min = 0, TGT_max = 256, TGT_zero = 128, 记为 Z

那么这个线性映射 F 是 Q_tgt = (X_src - SRC_zero) * (TGT_max - TGT_min)/(SRC_max - FP32，SRC_min) + TGT_zero  即 Q = X * 128/10^38 + 128, 
这样就把 源规范 的数值 X 映射成了 目标规范 范围内的数值。

一般来说 源规范 和 目标规范都是 关于0对称的，故零点都是0，且 min 和 max 关于 0 对称。那么 Q_tgt = X_src * TGT_max / SRC_max 

这个线性映射的系数，被称为 缩放因子 Scale Factor S。它的倒数是 逆映射dequantization 的系数。

step2: 数据类型转换，映射后的数值坐落在了目标规范的range内，故可以用目标规范的bits表示。
数据类型转换是发生实质机器内存操作，故它是真正损失精度的地方（高bits浮点数用低bits表示），同时硬件原生支持很重要（硬件里实现了目标规范的数据类型，而不是软件模拟），
同时各种极端情况（舍入/上下溢）的处理也很重要。

### 静态量化 是一种 后训练优化技术：after训练、浮点权重已确定

In [ ]:
#TODO

In [ ]:
#TODO
寻找 Amax 以确定量化参数

对于 缩放因子 S = TGT_max / SRC_max，目标规范 TGT_max 没什么好说的，但是 源规范 SRC_max 是一个“数值利用率”很低的选择，因为大部分数值不会是均匀分布在 源规范 的range内，
所以 SRC_max 使用 经验性的 Amax 替代，以充分利用 源数据的范围空间

Amax 确定方法: 使用一小部分 极具代表性、未label的真实数据集，前向运行模型，过程中收集每一层的激活值和权重的统计信息（比如最大绝对值Amax或min/max等）
根据这些统计信息，确定每一层的量化参数（scale factor S 和 zero point Z）。这些量化参数是固定的，此为“静态”之含义。

In [ ]:
动态量化

#TODO


In [ ]:
量化在计算图中，以Q节点和DeQ节点的形式表示

对于已经加载好的 模型，如何表示其量化版本的计算图？
插入量化/反量化节点 (Quantize/Dequantize, QDQ)：
在模型图中，通常会在层输入之前插入一个量化操作，在层输出之后插入一个反量化操作（或在下一层的输入处量化）。  
这个反量化操作是必要的，因为它将整数结果转换回浮点数，以便与未量化的部分或需要浮点输入的层兼容，并进行后续的层间操作。  
但在实际硬件执行时，反量化操作通常是“概念性”的，计算会尽可能保持在整数域。


对于未加载的 模型，可以在加载时预先转换成 低比特数

### 量化感知 QAT 是一种 训练策略：在训练中模拟量化

In [ ]:
为了在训练阶段就考虑量化的影响，使模型对量化误差具有鲁棒性，业界发展出了 量化感知训练 (Quantization-Aware Training, QAT)。

QAT 的核心思想是：

在训练过程中模拟量化效应： QAT 不是直接将浮点数转换为整数并进行整数运算，而是在浮点运算的训练流程中，插入“伪量化”或“假量化”节点。
伪量化 (Fake Quantization) 的机制：
每个张量（包括权重和激活）首先被量化到低比特整数范围。
然后，立即将这些整数值反量化回浮点数（例如 FP32）。
模型在训练时，所有的计算仍然是浮点运算。
通过这种“量化再反量化”的模拟过程，梯度可以在训练过程中“感知”到由于量化而引入的误差。这使得模型能够调整其权重，以最小化这些误差，从而在最终被真正量化为整数时，保持更高的精度。
量化参数的更新： QAT 通常也会在训练过程中动态地更新量化参数（如缩放因子和零点），或者使用一些更高级的统计方法来确定它们，以更好地适应训练过程中数值范围的变化。
优点： QAT 训练出的模型在量化后，通常能比静态量化达到更高的精度，有时甚至能接近原始的 FP32 模型。
缺点： 训练过程更复杂，需要修改训练代码，并且训练时间可能会略微增加。

In [ ]:
Int8